In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
data_folder = 'att_hackathon/datasets'

In [ ]:
#_file = 'ML_DATASET_Hackathon_Supervised.csv'
_file = 'version_2.xls'

In [16]:
def load_file(path):
  df = pd.read_excel(path)
  return df

def select_cutoff(df, cutoff):
  df = df[df['percentile'] <= cutoff]
  return df

def get_non_stratifiable(df, count):
  counts = pd.DataFrame(df.Team.value_counts()).reset_index()
  non_stratifiable = list(counts[counts['Team'] < count]['index'])
  data_to_strat = df[~df.Team.isin(non_stratifiable)]
  extra = df[df.Team.isin(non_stratifiable)]
  return extra

def merge_non_stratified_with_train(x_train, y_train, extra):
    x_train = pd.concat([x_train,extra[['Problem_Abstract']]], axis=0)
    y_train = pd.concat([y_train,extra[['Team']]], axis=0)
    return x_train, y_train

In [8]:
x_train, x_test, y_train, y_test = train_test_split(data_to_strat[['Problem_Abstract']], 
                                                    data_to_strat[['Team']], 
                                                    test_size=0.1, 
                                                    stratify=data_to_strat[['Team']])

In [8]:
### EXPERIMENTING WITH TFIDF
vect = TfidfVectorizer(strip_accents='unicode',
                      stop_words='english',
                      ngram_range=(1,5))
vect.fit(x_train.fillna('').Problem_Abstract)
x_train = vect.transform(x_train.fillna('').Problem_Abstract)
x_test = vect.transform(x_test.fillna('').Problem_Abstract)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

TfidfVectorizer(ngram_range=(1, 5), stop_words='english',
                strip_accents='unicode')

### using embeddings

In [9]:
import numpy as np
emb_path = 'att_hackathon/data/embeddings_mpnet.npy'
embedding = np.load(emb_path)

In [10]:
x_train.shape[0] + x_test.shape[0]

4713

In [76]:
y = data[['Team']]

In [77]:
y_train, y_test = train_test_split(y, test_size=0.1, stratify=y['Team'])

In [79]:
y_train.shape[0] + y_test.shape[0]

4641

In [11]:
x_train = embedding[y_train.index]
x_test = embedding[y_test.index]

x_train = np.concatenate((x_train,embedding[extra['Problem_Abstract'].index]))
y_train = pd.concat([y_train,extra[['Team']]], axis=0)


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
y_train.shape

(4176,)

In [12]:
from catboost import CatBoostClassifier
#import xgboost as xgb
#classifier = xgb.XGBClassifier(n_jobs=8, n_estimators=100, verbosity=3)
classifier = CatBoostClassifier(task_type='GPU', iterations=500)
classifier.fit(x_train, y_train)

Learning rate set to 0.155463
0:	learn: 4.2985138	total: 517ms	remaining: 4m 18s
1:	learn: 4.0145238	total: 882ms	remaining: 3m 39s
2:	learn: 3.7803773	total: 1.23s	remaining: 3m 23s
3:	learn: 3.6324453	total: 1.61s	remaining: 3m 19s
4:	learn: 3.5139059	total: 1.97s	remaining: 3m 15s
5:	learn: 3.4192841	total: 2.35s	remaining: 3m 13s
6:	learn: 3.3376369	total: 2.69s	remaining: 3m 9s
7:	learn: 3.2703511	total: 3.06s	remaining: 3m 8s
8:	learn: 3.2141463	total: 3.41s	remaining: 3m 5s
9:	learn: 3.1645642	total: 3.78s	remaining: 3m 5s
10:	learn: 3.1209350	total: 4.17s	remaining: 3m 5s
11:	learn: 3.0825407	total: 4.54s	remaining: 3m 4s
12:	learn: 3.0458804	total: 5s	remaining: 3m 7s
13:	learn: 3.0138967	total: 5.46s	remaining: 3m 9s
14:	learn: 2.9816594	total: 5.97s	remaining: 3m 12s
15:	learn: 2.9542441	total: 6.36s	remaining: 3m 12s
16:	learn: 2.9264430	total: 6.83s	remaining: 3m 14s
17:	learn: 2.8980534	total: 7.31s	remaining: 3m 15s
18:	learn: 2.8715341	total: 7.77s	remaining: 3m 16s
19:

In [15]:
y_test.value_counts()

AttributeError: ignored

In [14]:
y_predicted = classifier.predict(x_test)
from sklearn.metrics import classification_report
print(classification_report(y_predicted, y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          19       0.00      0.00      0.00         0
          22       0.00      0.00      0.00         0
          23       0.00      0.00      0.00         0
          24       0.07      0.14      0.10        14
          25       0.00      0.00      0.00         0
          26       0.00    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
classifier.save("model_0.cbm")

In [81]:
classifier